In [82]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

### First website

In [2]:
urls = [f'https://www.kyureki.com/keiyaku/2023/{i}/' for i in range(1, 13)]

In [3]:
url = 'https://www.kyureki.com/keiyaku/2023/6/'
page = requests.get(url)
soup = BeautifulSoup(page.text, 'html')
table = soup.find('table')
table

<table class="table table-striped table-bordered table-sm text-nowrap">
<thead><td>名前</td><td>年俸（万）</td><td>差額</td><td>年齢</td><td>生涯年俸</td><td>ドラフト</td><td>年俸推移</td></thead><tbody>
<tr><td><a href="https://www.kyureki.com/player/8400/">村上宗隆</a></td><td>60000</td><td>38000</td><td>24歳</td><td>158020</td>
<td><a href="https://www.kyureki.com/draft/2017/">2017年ヤクルト1位</a></td>
<td><a href="https://www.kyureki.com/keiyaku/p8400/">村上宗隆の年俸推移</a></td></tr><tr><td><a href="https://www.kyureki.com/player/465/">山田哲人</a></td><td>50000</td><td>0</td><td>31歳</td><td>390720</td>
<td><a href="https://www.kyureki.com/draft/2010/">2010年ヤクルト1位</a></td>
<td><a href="https://www.kyureki.com/keiyaku/p465/">山田哲人の年俸推移</a></td></tr><tr><td><a href="https://www.kyureki.com/player/1053/">青木宣親</a></td><td>34000</td><td>1000</td><td>41歳</td><td>345000</td>
<td><a href="https://www.kyureki.com/draft/2003/">2003年ヤクルト4位</a></td>
<td><a href="https://www.kyureki.com/keiyaku/p1053/">青木宣親の年俸推移</a></td></tr><tr><td><a hr

In [13]:
# get titles
thead = table.find('thead')
title_elements = thead.find_all('td')
titles = [title.text.strip() for title in title_elements]
titles

['名前', '年俸（万）', '差額', '年齢', '生涯年俸', 'ドラフト', '年俸推移']

In [20]:
df = pd.DataFrame(columns = titles)
df

,名前,年俸（万）,差額,年齢,生涯年俸,ドラフト,年俸推移


In [21]:
# insert data
tbody = table.find('tbody')
column_data = tbody.find_all('tr')

In [26]:
for row in column_data:
    row_data = row.find_all('td')
    individual_row_data = [data.text.strip() for data in row_data[:-1]]

    salary_shift = row_data[-1].find('a')
    link = salary_shift['href']
    individual_row_data.append(link)
    
    length = len(df)
    df.loc[length] = individual_row_data

In [27]:
df

,名前,年俸（万）,差額,年齢,生涯年俸,ドラフト,年俸推移
0,村上宗隆,60000,38000,24歳,158020,2017年ヤクルト1位,https://www.kyureki.com/keiyaku/p8400/
1,山田哲人,50000,0,31歳,390720,2010年ヤクルト1位,https://www.kyureki.com/keiyaku/p465/
2,青木宣親,34000,1000,41歳,345000,2003年ヤクルト4位,https://www.kyureki.com/keiyaku/p1053/
3,ドミンゴ・サンタナ,26400,7100,31歳,92600,,https://www.kyureki.com/keiyaku/p139462/
4,サイスニード,19100,9600,31歳,60020,,https://www.kyureki.com/keiyaku/p239420/
...,...,...,...,...,...,...,...
226,濱田太貴,[],,23歳,,2018年ヤクルト4位,https://www.kyureki.com/keiyaku/p123841/
227,木須デソウザフェリペ,[],,24歳,,2017年オリックス育成4位,https://www.kyureki.com/keiyaku/p67415/
228,橋本星哉,[],,23歳,,2022年ヤクルト育成1位,https://www.kyureki.com/keiyaku/p122775/
229,坂本拓己,[],,19歳,,2022年ヤクルト4位,https://www.kyureki.com/keiyaku/p234675/


## Use another website
Some players have "[]" in their salary but looking at another site, they seem to have more data, including those who did not play in the first team (major league equiv. of NPB)

In [97]:
pitcher_url = 'https://sp.baseball.findfriends.jp/team/player/2006001/2023/pitcher/'
batter_url = 'https://sp.baseball.findfriends.jp/team/player/2006001/2023/batter'

In [98]:
page = requests.get(pitcher_url)
soup = BeautifulSoup(page.content, 'html.parser') 
# used .content since Japanese characters were displayed wiild with .text

In [99]:
# Table class found by inspecting website
# <table class="ResultTable02b tablesorter tablesorter-default" role="grid" style="transition-property: transform; transform-origin: 0px 0px; transform: translate(0px, 0px) translateZ(0px);">
table = soup.find('table', class_ = 'ResultTable02b')

In [49]:
table

<table class="ResultTable02b">
<thead>
<tr>
<th>
<a>背<br/>番<br/>号</a>
</th>
<th>
名<br/>前
</th>
<th>
<a>年<br/>俸</a>
</th>
<th data-sortinitialorder="asc">
<a>防<br/>御<br/>率</a>
</th>
<th>
<a>試<br/>合<br/>数</a>
</th>
<th>
<a>勝<br/>利</a>
</th>
<th>
<a>敗<br/>戦</a>
</th>
<th>
<a>投<br/>球<br/>回<br/>数</a>
</th>
<th>
<a>勝<br/>率</a>
</th>
<th>
<a>完<br/>投</a>
</th>
<th>
<a>完<br/>封</a>
</th>
<th>
<a>セ<br/>｜<br/>ブ</a>
</th>
<th>
<a>ホ<br/>｜<br/>ル<br/>ド</a>
</th>
<th>
<a>奪<br/>三<br/>振</a>
</th>
<th>
<a>奪<br/>三<br/>振<br/>率</a>
</th>
<th>
<a>失<br/>点</a>
</th>
<th>
<a>自<br/>責<br/>点</a>
</th>
<th>
<a>被<br/>安<br/>打</a>
</th>
<th>
<a>被<br/>本<br/>塁<br/>打</a>
</th>
<th>
<a>与<br/>四<br/>球</a>
</th>
<th>
<a>与<br/>死<br/>球</a>
</th>
<th>
<a>与<br/>四<br/>球<br/>率</a>
</th>
</tr>
</thead>
<tbody>
<tr>
<th>
<a href="https://sp.baseball.findfriends.jp/team/uniform/2006001/12/">12</a>
</th>
<th class="name">
<a href="https://sp.baseball.findfriends.jp/player/19880036/">石山泰稚</a>
</th>
<td class="score">
15000
</td>
<td cla

In [100]:
# get titles
thead = table.find('thead')
title_elements = thead.find_all('th')
titles = [title.text.strip().replace('｜', 'ー') for title in title_elements]
titles

['背番号',
 '名前',
 '年俸',
 '防御率',
 '試合数',
 '勝利',
 '敗戦',
 '投球回数',
 '勝率',
 '完投',
 '完封',
 'セーブ',
 'ホールド',
 '奪三振',
 '奪三振率',
 '失点',
 '自責点',
 '被安打',
 '被本塁打',
 '与四球',
 '与死球',
 '与四球率']

In [101]:
df = pd.DataFrame(columns = titles)
df

,背番号,名前,年俸,防御率,試合数,勝利,敗戦,投球回数,勝率,完投,...,ホールド,奪三振,奪三振率,失点,自責点,被安打,被本塁打,与四球,与死球,与四球率


In [102]:
# insert data
tbody = table.find('tbody')
column_data = tbody.find_all('tr')

In [103]:
for row in column_data:
    row_data = row.find_all(['th', 'td'])
    individual_row_data = [data.text.strip() for data in row_data]
    # print(individual_row_data)
    length = len(df)
    df.loc[length] = individual_row_data

In [104]:
df.head()

,背番号,名前,年俸,防御率,試合数,勝利,敗戦,投球回数,勝率,完投,...,ホールド,奪三振,奪三振率,失点,自責点,被安打,被本塁打,与四球,与死球,与四球率
0,12,石山泰稚,15000,4.40,50,3,6,47,.333,0,...,24,41,7.85,27,23,51,3,13,3,2.49
1,012,近藤弘樹,900,--.--,-,-,-,-,.---,-,...,-,-,--.--,-,-,-,-,-,-,--.--
2,14,高梨裕稔,4300,4.69,15,0,3,40 1/3,.000,0,...,1,30,6.69,23,21,42,6,10,1,2.23
3,014,鈴木裕太,300,--.--,-,-,-,-,.---,-,...,-,-,--.--,-,-,-,-,-,-,--.--
4,15,山下輝,1400,--.--,-,-,-,-,.---,-,...,-,-,--.--,-,-,-,-,-,-,--.--


In [105]:
# convert numeric columns' data types
num_cols = df.columns[2:]
df[num_cols] = df[num_cols].apply(pd.to_numeric, errors='coerce')
df.head()

,背番号,名前,年俸,防御率,試合数,勝利,敗戦,投球回数,勝率,完投,...,ホールド,奪三振,奪三振率,失点,自責点,被安打,被本塁打,与四球,与死球,与四球率
0,12,石山泰稚,15000,4.40,50.0,3.0,6.0,47.0,0.333,0.0,...,24.0,41.0,7.85,27.0,23.0,51.0,3.0,13.0,3.0,2.49
1,012,近藤弘樹,900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,14,高梨裕稔,4300,4.69,15.0,0.0,3.0,NaN,0.000,0.0,...,1.0,30.0,6.69,23.0,21.0,42.0,6.0,10.0,1.0,2.23
3,014,鈴木裕太,300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,15,山下輝,1400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Function to create DataFrame (combining above codes)

In [96]:
def create_player_data_df(url: str) -> pd.DataFrame:
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    table = soup.find('table', class_ = 'ResultTable02b')

    # get titles
    thead = table.find('thead')
    title_elements = thead.find_all('th')
    titles = [title.text.strip().replace('｜', 'ー') for title in title_elements]
    df = pd.DataFrame(columns = titles)
    
    # add rows
    tbody = table.find('tbody')
    column_data = tbody.find_all('tr')
    for row in column_data:
        row_data = row.find_all(['th', 'td'])
        individual_row_data = [data.text.strip() for data in row_data]
        # print(individual_row_data)
        length = len(df)
        df.loc[length] = individual_row_data

    # convert numeric columns' data types
    num_cols = columns[2:]
    df[num_cols] = df[num_cols].apply(pd.to_numeric, errors='coerce')

    return df

## Getting URLs

In [132]:
url = 'https://sp.baseball.findfriends.jp/?pid=db'
player_page = requests.get(url)
player_soup = BeautifulSoup(player_page.content, 'html.parser') 

In [133]:
player_pg_urls = player_soup.find_all('ul')
player_pg_urls

[<ul class="fc">
 <li>
 <a href="https://sp.baseball.findfriends.jp/?from=navi" title="週刊ベースボールONLINEトップ">トップ</a>
 </li>
 <li>
 <a href="https://column.sp.baseball.findfriends.jp/?pid=top&amp;from=navi" title="週刊ベースボールニュース＆コラム">ニュース＆コラム</a>
 </li>
 <li>
 <a href="https://sp.baseball.findfriends.jp/?pid=game_flash_list&amp;from=navi" title="プロ野球試合速報">試合速報</a>
 </li>
 <li class="Active">
 <a href="https://sp.baseball.findfriends.jp/?pid=db" title="選手データ">選手データ</a>
 </li>
 <li>
 <a href="https://sp.baseball.findfriends.jp/?pid=baseball_special&amp;from=navi" title="野球特集・連載コラム一覧">特集</a>
 </li>
 <li>
 <a href="https://sp.baseball.findfriends.jp/?pid=baseball_season&amp;from=navi" title="プロ野球特集">プロ野球</a>
 </li>
 <li>
 <a href="https://sp.baseball.findfriends.jp/?pid=baseball_amateur&amp;from=navi" title="アマチュア野球特集（高校野球･大学野球･社会人野球）">アマチュア</a>
 </li>
 <li>
 <a href="https://sp.baseball.findfriends.jp/?pid=draft&amp;from=navi" title="プロ野球ドラフト会議特集">ドラフト</a>
 </li>
 <li>
 <a href="https://sp.base

In [141]:
team_ul_elements = player_soup.find_all('ul', class_ = 'db_team_select_box')
team_ul_elements

[<ul class="db_team_select_box">
 <li>
 セ･リーグ
 </li>
 <li>
 <a href="/team/player/1968001/">広島</a>
 </li>
 <li>
 <a href="/team/player/1947001/">巨人</a>
 </li>
 <li>
 <a href="/team/player/2012001/">DeNA</a>
 </li>
 <li>
 <a href="/team/player/1961001/">阪神</a>
 </li>
 <li>
 <a href="/team/player/2006001/">ヤクルト</a>
 </li>
 <li>
 <a href="/team/player/1954001/">中日</a>
 </li>
 </ul>,
 <ul class="db_team_select_box">
 <li>
 パ･リーグ
 </li>
 <li>
 <a href="/team/player/2004001/">日本ハム</a>
 </li>
 <li>
 <a href="/team/player/2005001/">ソフトバンク</a>
 </li>
 <li>
 <a href="/team/player/1992001/">ロッテ</a>
 </li>
 <li>
 <a href="/team/player/2008001/">西武</a>
 </li>
 <li>
 <a href="/team/player/2005003/">楽天</a>
 </li>
 <li>
 <a href="/team/player/2005002/">オリックス</a>
 </li>
 </ul>]

In [150]:
team_dict = {}

for team_ul in team_ul_elements:
    league_name = team_ul.find('li').text.strip()
    team_links = [a['href'] for a in team_ul.find_all('a')]
    team_names = [a.text for a in team_ul.find_all('a')]

    for link, name in zip(team_links, team_names):
        team_dict[name] = {
            'url': link,
            'league': 'セ' if league_name == 'セ･リーグ' else 'パ'
        }

team_dict

{'広島': {'url': '/team/player/1968001/', 'league': 'セ'},
 '巨人': {'url': '/team/player/1947001/', 'league': 'セ'},
 'DeNA': {'url': '/team/player/2012001/', 'league': 'セ'},
 '阪神': {'url': '/team/player/1961001/', 'league': 'セ'},
 'ヤクルト': {'url': '/team/player/2006001/', 'league': 'セ'},
 '中日': {'url': '/team/player/1954001/', 'league': 'セ'},
 '日本ハム': {'url': '/team/player/2004001/', 'league': 'パ'},
 'ソフトバンク': {'url': '/team/player/2005001/', 'league': 'パ'},
 'ロッテ': {'url': '/team/player/1992001/', 'league': 'パ'},
 '西武': {'url': '/team/player/2008001/', 'league': 'パ'},
 '楽天': {'url': '/team/player/2005003/', 'league': 'パ'},
 'オリックス': {'url': '/team/player/2005002/', 'league': 'パ'}}

In [ ]:
# combined into function
def create_team_dict():
    url = 'https://sp.baseball.findfriends.jp/?pid=db'
    player_page = requests.get(url)
    player_soup = BeautifulSoup(player_page.content, 'html.parser')
    team_ul_elements = player_soup.find_all('ul', class_ = 'db_team_select_box')

    team_dict = {}

    for team_ul in team_ul_elements:
        league_name = team_ul.find('li').text.strip()
        team_links = [a['href'] for a in team_ul.find_all('a')]
        team_names = [a.text for a in team_ul.find_all('a')]
    
        for link, name in zip(team_links, team_names):
            team_dict[name] = {
                'url': link,
                'league': 'セ' if league_name == 'セ･リーグ' else 'パ'
            }
    
    return team_dict